In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import DensityMatrix
from qiskit_aer.quantum_info import AerDensityMatrix
from qiskit.circuit.library import QuantumVolume

## 1 quibit circuits

In [ ]:
qc = QuantumVolume(1, seed=1111)
%timeit -o DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(1, seed=1111)
%timeit -o AerDensityMatrix(qc)

## 2 quibits circuits

In [ ]:
qc = QuantumVolume(2, seed=1111)
%timeit -o DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(2, seed=1111)
%timeit -o AerDensityMatrix(qc)

## More quibits circuits

In [ ]:
import psutil
import numpy as np

mem = psutil.virtual_memory()
# assume 'sys.getsizeof(1j) * 2^(2*n) <= mem.free' is safe
# 32 * 2^(2*n) <= mem.free
max_qubits = int(np.log2(mem.free / sys.getsizeof(1j)) // 2)
print(f'{max_qubits=}')
n_qubits_range = range(1, max_qubits+1)

In [ ]:
if False:
    qc = QuantumVolume(13, seed=1111)
    %timeit -n 1 -r 1 DensityMatrix(qc) #=> 7min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

if False:
    qc = QuantumVolume(13, seed=1111)
    %timeit -n 1 -r 1 AerDensityMatrix(qc) #=> 2min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

if False:
    qc = QuantumVolume(14, seed=1111)
    %timeit -n 1 -r 1 DensityMatrix(qc) #=> 36min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

if False:
    qc = QuantumVolume(14, seed=1111)
    %timeit -n 1 -r 1 AerDensityMatrix(qc) #=> 13min 28s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [ ]:
results_dm = []
results_adm = []

for n_qubits in n_qubits_range:
    qc = QuantumVolume(n_qubits, seed=1111)
    if n_qubits <= 10:
        result = %timeit -o DensityMatrix(qc)
        results_dm.append([result.average, result.stdev])
        result = %timeit -o AerDensityMatrix(qc)
        results_adm.append([result.average, result.stdev])
    else:
        result = %timeit -n 1 -r 1 -o DensityMatrix(qc)
        results_dm.append([result.average, result.stdev])
        result = %timeit -n 1 -r 1 -o AerDensityMatrix(qc)
        results_adm.append([result.average, result.stdev])

In [ ]:
import pickle

with open('results_dm.pkl', 'wb') as fout:
    pickle.dump(results_dm, fout)
with open('results_adm.pkl', 'wb') as fout:
    pickle.dump(results_adm, fout)

In [ ]:
import pickle

with open('results_dm.pkl', 'rb') as fin:
    results_dm = pickle.load(fin)
with open('results_adm.pkl', 'rb') as fin:
    results_adm = pickle.load(fin)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_qubits_range = range(1, len(results_dm)+1)

if isinstance(results_dm, list):
    results_dm = np.array(results_dm)
if isinstance(results_adm, list):
    results_adm = np.array(results_adm)

fig, ax = plt.subplots()
ax.errorbar(x=n_qubits, y=results_dm[:, 0], yerr=results_dm[:, 1], fmt='-o', color='b')
ax.errorbar(x=n_qubits, y=results_adm[:, 0], yerr=results_adm[:, 1], fmt='-o', color='r')
ax.set_xlabel('num of qubits')
ax.set_ylabel('elapsed mean/stdev time [s]')
ax.set_title('Performance comparison between DensityMatrix and AerDensityMatrix')
ax.legend(['DensityMatrix', 'AerDensityMatrix'])
plt.grid()
plt.show()